In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library:
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions:
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

# Документация:
- https://scikit-learn.ru/1-1-linear-models/

In [2]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [12]:
df = pd.read_csv('../datasets/insurance_miptstats.csv', parse_dates=[0])
df.head()

,birthday,sex,bmi,children,smoker,region,charges
0,2001-12-20,female,27.900,0,yes,southwest,16884.92400
1,2003-03-18,male,33.770,1,no,southeast,1725.55230
2,1992-11-02,male,33.000,3,no,southeast,4449.46200
3,1987-07-27,male,22.705,0,no,northwest,21984.47061
4,1988-11-04,male,28.880,0,no,northwest,3866.85520


In [13]:
df.dtypes

birthday    datetime64[ns]
sex                 object
bmi                float64
children             int64
smoker              object
region              object
charges            float64
dtype: object

In [17]:
from datetime import datetime

df['age'] = (pd.Timestamp(datetime.now()) - df['birthday']) / pd.Timedelta(365, 'd')

#df['age'] = datetime.now().year - df.birthday.year - ((datetime.now().month, datetime.now().day) < (df.birthday.month, df.birthday.day))

In [18]:
df.head()

,birthday,sex,bmi,children,smoker,region,charges,age
0,2001-12-20,female,27.900,0,yes,southwest,16884.92400,22.729998
1,2003-03-18,male,33.770,1,no,southeast,1725.55230,21.488902
2,1992-11-02,male,33.000,3,no,southeast,4449.46200,31.866984
3,1987-07-27,male,22.705,0,no,northwest,21984.47061,37.140957
4,1988-11-04,male,28.880,0,no,northwest,3866.85520,35.864245


In [19]:
real_features= ['age', 'bmi', 'children']
categorial_features = ['sex', 'smoker', 'region']

In [23]:
train, test = train_test_split(df, test_size=0.25)

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
OneHotEncoder?

Init signature:
OneHotEncoder(
    *,
    categories='auto',
    drop=None,
    sparse_output=True,
    dtype=<class 'numpy.float64'>,
    handle_unknown='error',
    min_frequency=None,
    max_categories=None,
    feature_name_combiner='concat',
)
Docstring:     
Encode categorical features as a one-hot numeric array.

The input to this transformer should be an array-like of integers or
strings, denoting the values taken on by categorical (discrete) features.
The features are encoded using a one-hot (aka 'one-of-K' or 'dummy')
encoding scheme. This creates a binary column for each category and
returns a sparse matrix or dense array (depending on the ``sparse_output``
parameter).

By default, the encoder derives the categories based on the unique values
in each feature. Alternatively, you can also specify the `categories`
manually.

This encoding is needed for feeding categorical data to many scikit-learn
estimators, notably linear models and SVMs with the standard kernels.

Note: a o

In [25]:
encoder = OneHotEncoder(drop='first', sparse_output=False)
train_cat = encoder.fit_transform(train[categorial_features])
train_cat

array([[1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0.]])

In [26]:
encoder.categories_

[array(['female', 'male'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['northeast', 'northwest', 'southeast', 'southwest'], dtype=object)]

In [28]:
import numpy as np
x_train = np.hstack([train[real_features], train_cat])
x_train.shape

(1003, 8)

In [31]:
target_features = ['charges']

In [30]:
from sklearn.linear_model import LinearRegression
LinearRegression?

Init signature:
LinearRegression(
    *,
    fit_intercept=True,
    copy_X=True,
    n_jobs=None,
    positive=False,
)
Docstring:     
Ordinary least squares Linear Regression.

LinearRegression fits a linear model with coefficients w = (w1, ..., wp)
to minimize the residual sum of squares between the observed targets in
the dataset, and the targets predicted by the linear approximation.

Parameters
----------
fit_intercept : bool, default=True
    Whether to calculate the intercept for this model. If set
    to False, no intercept will be used in calculations
    (i.e. data is expected to be centered).

copy_X : bool, default=True
    If True, X will be copied; else, it may be overwritten.

n_jobs : int, default=None
    The number of jobs to use for the computation. This will only provide
    speedup in case of sufficiently large problems, that is if firstly
    `n_targets > 1` and secondly `X` is sparse or if `positive` is set
    to `True`. ``None`` means 1 unless in a
    :obj:`

In [33]:
lin_r = LinearRegression()
lin_r.fit(x_train, train[target_features])

LinearRegression()

In [34]:
lin_r.coef_

array([[  257.27022648,   321.54113831,   356.22197665,  -434.24932107,
        24135.85715571,   -92.57861687,  -992.27264474, -1153.09486534]])

In [35]:
lin_r.intercept_

array([-12243.65257218])

In [36]:
test_cat = encoder.transform(test[categorial_features])
test_cat

array([[0., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       ...,
       [1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.]])

In [37]:
x_test = np.hstack([test[real_features], test_cat])
x_test.shape

(335, 8)

In [38]:
y_pred = lin_r.predict(x_test)

In [41]:

np.sqrt(((test[target_features]-y_pred) ** 2).mean())

charges    6373.327406
dtype: float64

In [45]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test[target_features], y_pred)
np.sqrt(mse)

6373.327406470887

# Еще один пример:
- статистика стоимости недвижимости в Бостоне: https://pythonru.com/uroki/linear-regression-sklearn 